In [1]:
import torch
from peft.utils.save_and_load import load_peft_weights

Below, we show how to load a model and flatten the weights to create a datapoint for the dataset of model weights. You will have to loop this over all the fine-tuned model weights and concatenate them along the first dimension to obtain an mxn tensor, where there are m models and n is the dimensionality of the flattened LoRA weights. In order to have a one-to-one correspondence for each model with the dataset of weights we provide, you will have to first sort the models in increasing alphanumeric ordering based on the folder name it was trained on, then concatenate them in that order. You can find the labeled attributes for each set of images using the ``identity_df.pt`` that we provide, and indexing into the ``file`` attribute, which should correspond to the number in the folder name.

In [2]:

adapters_weights = load_peft_weights("models/0/unet", device="cuda:0")


In [3]:
w = []
for key in adapters_weights.keys():
    w.append(adapters_weights[key].flatten())
w = torch.cat(w, 0).unsqueeze(0)
print(w.shape)



torch.Size([1, 99648])


# PCA Utils

In [4]:
def pca(a):
    mean = torch.mean(a, dim=0)
    std = torch.std(a, dim=0)
    scaled_data = (a - mean)/std
    u,s,v = torch.svd(scaled_data)
    return u,s,v, mean, std

### Example use of PCA function

flat_weights is a mxn dimensional tensor, where m is the number of models, and n is the dimensionality of the flattened LoRA weights. To obtain flat_weights, you have to run the flattening code from above over all the models and then concatenate them over the first dimension to get the mxn tensor. If you run out of memory during PCA, replace ``u,s,v = torch.svd(scaled_data)`` in the pca function with:  <br>
``u,s,v = torch.svd_lowrank(flat_weights, q=10000, niter=20, M=None)``.

In [7]:
u,s,v, mean, std = pca(flat_weights) 

Below, we provide example code of how to project the original weights onto the first k principal components and unproject back to the original LoRA weight space. 

In [8]:
def project(a,v,k, mean, std):
    data = (a-mean)/std
    new = torch.matmul(data,v[:, :k])
    return new

In [9]:
def unproject(projected,v,k, mean, std):
    new = torch.matmul(projected,v[:, :k].T)
    new = new*std+mean
    return new

In [ ]:
projection = project(w, v, 10000, mean std)
unprojection = unproject(projection, v, 10000, mean, std)